<a href="https://colab.research.google.com/github/michaelwty/-/blob/master/Update_Sorting_Comparison_7_17_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing sort methods 
### By Jeff Hale
## See [this Medium article](https://towardsdatascience.com/surprising-sorting-tips-for-data-scientists-9c360776d7e?source=friends_link&sk=c11b328e381e6c95b8e75d58b09546a3) for discussion.

In [0]:
!pip3 install torch torchvision
!pip3 install tensorflow-gpu==2.0.0-beta1 #this install GPU capabilities, but actually shows down sorts use #tensorflow==2.0.0-beta1 instead

In [0]:
import numpy as np
import pandas as pd
import torch
import tensorflow as tf

!pip list

Package                  Version              
------------------------ ---------------------
absl-py                  0.7.1                
alabaster                0.7.12               
albumentations           0.1.12               
altair                   3.1.0                
astor                    0.8.0                
astropy                  3.0.5                
atari-py                 0.1.15               
atomicwrites             1.3.0                
attrs                    19.1.0               
audioread                2.1.8                
autograd                 1.2                  
Babel                    2.7.0                
backcall                 0.1.0                
backports.tempfile       1.0                  
backports.weakref        1.0.post1            
beautifulsoup4           4.6.3                
bleach                   3.1.0                
blis                     0.2.4                
bokeh                    1.0.4                
boto         

In [0]:
torch.cuda.is_available()

True

In [0]:
print("GPU Available: ", tf.test.is_gpu_available())

GPU Available:  True


In [0]:
tf.debugging.set_log_device_placement(True)

In [0]:
!nvidia-smi

Wed Jul 17 15:11:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    70W / 149W |    333MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import sys
sys.version

'3.6.8 (default, Jan 14 2019, 11:02:34) \n[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'

In [0]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: G

Make work reproducible.

In [0]:
seed = 34

# python 
import random
random.seed(seed)

# numpy 
np.random.seed(seed)

# pytorch 
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# tensorflow 2.0
tf.random.set_seed(seed)

In [0]:
!cat /proc/meminfo

MemTotal:       13335276 kB
MemFree:         2825016 kB
MemAvailable:   11337200 kB
Buffers:          130924 kB
Cached:          8054408 kB
SwapCached:            0 kB
Active:          2580332 kB
Inactive:        7362556 kB
Active(anon):    1422244 kB
Inactive(anon):     3420 kB
Active(file):    1158088 kB
Inactive(file):  7359136 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1800 kB
Writeback:             0 kB
AnonPages:       1757612 kB
Mapped:           684672 kB
Shmem:              3980 kB
Slab:             361984 kB
SReclaimable:     320696 kB
SUnreclaim:        41288 kB
KernelStack:        3872 kB
PageTables:        10160 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667636 kB
Committed_AS:    3404756 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

## Data
Create 1M random data points between 0 and 1000 in a numpy array.


In [0]:
my_numpy_array = np.random.randint(1000, size=(1000000))
my_numpy_array

array([417, 122, 490, ..., 768, 269, 551])

Make a Pandas Series from the array.

In [0]:
my_pandas_series = pd.Series(my_numpy_array)
my_pandas_series.tail()

999995    597
999996    282
999997    768
999998    269
999999    551
dtype: int64

Make a vanilla Python list.

In [0]:
my_python_list = list(my_numpy_array)
my_python_list[-5:]

[597, 282, 768, 269, 551]

Make a PyTorch tensor.

In [0]:
my_pytorch_tensor = torch.from_numpy(my_numpy_array)
my_pytorch_tensor[-5:]

tensor([597, 282, 768, 269, 551])

Make a TensorFlow tensor.

In [0]:
my_tf_tensor = tf.convert_to_tensor(my_numpy_array)

my_tf_tensor

<tf.Tensor: id=0, shape=(1000000,), dtype=int64, numpy=array([417, 122, 490, ..., 768, 269, 551])>

# Sorting

Numpy sorting with copy

In [0]:
%time np.sort(my_numpy_array) #quicksort

CPU times: user 60.8 ms, sys: 263 µs, total: 61.1 ms
Wall time: 62.9 ms


array([  0,   0,   0, ..., 999, 999, 999])

In [0]:
%time np.sort(my_numpy_array, kind='mergesort')

CPU times: user 82.7 ms, sys: 7.75 ms, total: 90.5 ms
Wall time: 96.4 ms


array([  0,   0,   0, ..., 999, 999, 999])

In [0]:
%time np.sort(my_numpy_array, kind='heapsort')

CPU times: user 180 ms, sys: 2.78 ms, total: 183 ms
Wall time: 185 ms


array([  0,   0,   0, ..., 999, 999, 999])

### Pandas sorting with copy

In [0]:
%time my_pandas_series.sort_values()

CPU times: user 184 ms, sys: 28.1 ms, total: 212 ms
Wall time: 220 ms


891530      0
694680      0
630467      0
166018      0
77652       0
864908      0
557435      0
694698      0
748195      0
309941      0
865475      0
257029      0
164797      0
26961       0
325617      0
128557      0
494571      0
23127       0
765953      0
417573      0
695008      0
375687      0
257573      0
646949      0
434237      0
630628      0
427515      0
647010      0
100065      0
364683      0
         ... 
677760    999
68873     999
368481    999
918489    999
298885    999
794740    999
70249     999
794858    999
100728    999
502383    999
645807    999
591534    999
534672    999
942603    999
567690    999
756143    999
853068    999
756740    999
575767    999
147953    999
795451    999
368866    999
852789    999
100402    999
368736    999
700505    999
591659    999
368616    999
963258    999
520847    999
Length: 1000000, dtype: int64

In [0]:
%time my_pandas_series.sort_values(kind='mergesort')

CPU times: user 550 ms, sys: 12.2 ms, total: 562 ms
Wall time: 573 ms


100         0
125         0
1766        0
2450        0
5039        0
5163        0
6195        0
6774        0
7451        0
7698        0
7899        0
8153        0
8934        0
10373       0
10880       0
11239       0
11385       0
14874       0
16031       0
16636       0
18375       0
18604       0
20755       0
21192       0
21606       0
22197       0
22300       0
22365       0
23127       0
23831       0
         ... 
971829    999
972668    999
973941    999
974619    999
974633    999
974951    999
975385    999
975486    999
979417    999
980045    999
980156    999
980630    999
981501    999
982942    999
983133    999
984186    999
987200    999
988014    999
988637    999
989206    999
989307    999
989876    999
990170    999
990915    999
990961    999
991536    999
992483    999
993914    999
999709    999
999755    999
Length: 1000000, dtype: int64

In [0]:
%time my_pandas_series.sort_values(kind='heapsort')

CPU times: user 477 ms, sys: 9.77 ms, total: 487 ms
Wall time: 494 ms


529516      0
653305      0
513651      0
891530      0
558216      0
567343      0
850299      0
58269       0
778536      0
239409      0
436390      0
467280      0
980549      0
884415      0
235493      0
979973      0
92793       0
432277      0
56430       0
240660      0
745545      0
881510      0
519023      0
990306      0
10880       0
565794      0
282882      0
812679      0
782800      0
694913      0
         ... 
270595    999
540911    999
540490    999
539802    999
134792    999
134735    999
269364    999
538386    999
537409    999
268248    999
268024    999
133677    999
534672    999
267297    999
534303    999
534211    999
266459    999
531340    999
529594    999
528874    999
528582    999
527938    999
16496     999
263846    999
527646    999
263778    999
263045    999
525665    999
525603    999
262665    999
Length: 1000000, dtype: int64

Now in place.

### Numpy inplace

In [0]:
%time my_numpy_array.sort() # quicksort

CPU times: user 59.2 ms, sys: 2.62 ms, total: 61.8 ms
Wall time: 64.9 ms


In [0]:
%time my_numpy_array.sort(kind='mergesort')

CPU times: user 39.7 ms, sys: 955 µs, total: 40.7 ms
Wall time: 40.6 ms


In [0]:
%time my_numpy_array.sort(kind='heapsort')

CPU times: user 74.8 ms, sys: 982 µs, total: 75.8 ms
Wall time: 78.8 ms


Wow, quicksort and heapsort are way faster inplace, but mergesort is slower. wierd.

### Pandas inplace

In [0]:
%time my_pandas_series.sort_values(inplace=True) #quicksort

CPU times: user 57.8 ms, sys: 9.18 ms, total: 67 ms
Wall time: 67.8 ms


In [0]:
%time my_pandas_series.sort_values(kind='mergesort',  inplace=True)

CPU times: user 85.3 ms, sys: 15.8 ms, total: 101 ms
Wall time: 103 ms


In [0]:
%time my_pandas_series.sort_values(kind='heapsort',  inplace=True)

CPU times: user 142 ms, sys: 6.32 ms, total: 149 ms
Wall time: 152 ms


Slightly faster inplace than when copying with pandas, but not way faster. Mergesort with pandas actually way faster than numpy. Why?

In [0]:
#timeit one time milliseconds for 1M data points between 0 and 999
timings_dict = dict(
    pd_inplace_quicksort=68,
    pd_inplace_mergesort=103,
    pd_inplace_heapsort=152,
    pd_copy_quicksort=220,
    pd_copy_mergesort=573,
    pd_copy_heapsort=494,
    np_inplace_quicksort=65,
    np_inplace_mergesort=41,
    np_inplace_heapsort=79,
    np_copy_quicksort=63,
    np_copy_mergesort=96,
    np_copy_heapsort=185
)

## Vanilla Python sorting
inplace

In [0]:
%time my_python_list.sort()

CPU times: user 616 ms, sys: 5.02 ms, total: 621 ms
Wall time: 625 ms


In [0]:
%time sorted(my_python_list)

CPU times: user 176 ms, sys: 1.75 ms, total: 178 ms
Wall time: 179 ms


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [0]:
timings_dict['python_inplace']=625
timings_dict['python_copy']=179

## TF sorting

Using CPU


In [0]:
with tf.device('/CPU:0'):
  %time tf.sort(my_tf_tensor)

Executing op Neg in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TopKV2 in device /job:localhost/replica:0/task:0/device:CPU:0
CPU times: user 75.7 ms, sys: 2.4 ms, total: 78.1 ms
Wall time: 87.7 ms


In [0]:
timings_dict['tf_cpu']=88

Using GPU

In [0]:
with tf.device('/GPU:0'):
  %time tf.sort(my_tf_tensor)

Executing op Neg in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TopKV2 in device /job:localhost/replica:0/task:0/device:GPU:0
CPU times: user 424 ms, sys: 137 ms, total: 561 ms
Wall time: 569 ms


In [0]:
timings_dict['tf_gpu']=569

## PyTorch sorting

Using CPU

In [0]:
%time torch.sort(my_pytorch_tensor)

CPU times: user 48.7 ms, sys: 12.7 ms, total: 61.5 ms
Wall time: 60.7 ms


torch.return_types.sort(values=tensor([  0,   0,   0,  ..., 999, 999, 999]), indices=tensor([   674,    672,    678,  ..., 999343, 999342, 999999]))

In [0]:
timings_dict['pytorch_cpu']=61

Now PyTorch with GPU enabled

In [0]:
gpu_tensor=my_pytorch_tensor.cuda()
%time torch.sort(gpu_tensor)

CPU times: user 2.16 ms, sys: 4.02 ms, total: 6.18 ms
Wall time: 6.15 ms


torch.return_types.sort(values=tensor([  0,   0,   0,  ..., 999, 999, 999], device='cuda:0'), indices=tensor([     0,      1,      2,  ..., 999997, 999998, 999999], device='cuda:0'))

In [0]:
timings_dict['pytorch_gpu']=6

In [0]:
timings_dict

{'np_copy_heapsort': 185,
 'np_copy_mergesort': 96,
 'np_copy_quicksort': 63,
 'np_inplace_heapsort': 79,
 'np_inplace_mergesort': 41,
 'np_inplace_quicksort': 65,
 'pd_copy_heapsort': 494,
 'pd_copy_mergesort': 573,
 'pd_copy_quicksort': 220,
 'pd_inplace_heapsort': 152,
 'pd_inplace_mergesort': 103,
 'pd_inplace_quicksort': 68,
 'python_copy': 179,
 'python_inplace': 625,
 'pytorch_cpu': 61,
 'pytorch_gpu': 6,
 'tf_cpu': 88,
 'tf_gpu': 569}